## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config3 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,andenes,NO,2021-08-15 19:20:35,69.3143,16.1194,49.98,87,75,27.63,broken clouds,0.0,0
1,1,bluff,NZ,2021-08-15 19:20:35,-46.6000,168.3333,52.81,70,97,19.69,overcast clouds,0.0,0
2,2,pangnirtung,CA,2021-08-15 19:20:36,66.1451,-65.7125,46.40,87,75,13.80,broken clouds,0.0,0
3,3,mataura,NZ,2021-08-15 19:20:36,-46.1927,168.8643,52.48,90,100,5.21,overcast clouds,0.0,0
4,4,vaini,TO,2021-08-15 19:20:36,-21.2000,-175.2000,66.36,77,40,8.05,scattered clouds,0.0,0


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_bool = input("Do you want it to be raining? (yes/no) ")
snow_bool = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 66
What is the maximum temperature you would like for your trip? 88
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) yes


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
if rain_bool == "no" and snow_bool == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] == 0)]
elif rain_bool == "no" and snow_bool == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] == 0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]
elif rain_bool == "yes" and snow_bool == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] == 0.0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Rain (inches)"] >= 0.0) & \
                                        (city_data_df["Snow (inches)"] >= 0.0)]

KeyError: 'Rain (inches)'

In [5]:
# 4a. Determine if there are any empty rows.
clean_df = filtered_cities_df.dropna()

NameError: name 'filtered_cities_df' is not defined

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

NameError: name 'clean_df' is not defined

In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

NameError: name 'hotel_df' is not defined

In [8]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

NameError: name 'hotel_df' is not defined

In [9]:
# 8a. Create the output File (CSV)
output_data_file = "../Weather_Database/Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label = "City_ID")

NameError: name 'hotel_df' is not defined

In [10]:

gmaps.configure(api_key= g_key)

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

NameError: name 'hotel_df' is not defined